# MRJob Test DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  4
* **Date:**  5/31/16

In [1]:
%%writefile mr_wc.py
"""An implementation of wc as an MRJob.
This is meant as an example of why mapper_final is useful."""
from mrjob.job import MRJob

class MRWordCountUtility(MRJob):
    def __init__(self, *args, **kwargs):
        super(MRWordCountUtility, self).__init__(*args, **kwargs)
        self.chars=0
        self.words=0
        self.lines=0
    def mapper(self, _, line):
        # Don't actually yield anything for each
        # line. Instead, collect them and yield the
        # sums when all lines have been processed.
        # The results will be collected by the
        # reducer.
        self.chars+=len(line) + 1 # +1 for newline
        self.words+=sum(1 for word in line.split() if word.strip())
        self.lines+=1
    def mapper_final(self):
        yield('chars',self.chars)
        yield('words',self.words)
        yield('lines',self.lines)
    def reducer(self, key, values):
        yield(key, sum(values))
        
if __name__ == '__main__':
    MRWordCountUtility.run()

Writing mr_wc.py


In [2]:
!python mr_wc.py mr_wc.py

No configs found; falling back on auto-configuration
Creating temp directory /tmp/mr_wc.hadoop.20160531.224159.249167
Running step 1 of 1...
Streaming final output from /tmp/mr_wc.hadoop.20160531.224159.249167/output...
"words"	102
"chars"	987
"lines"	28
Removing temp directory /tmp/mr_wc.hadoop.20160531.224159.249167...


In [3]:
from mr_wc import MRWordCountUtility

mr_job= MRWordCountUtility(args=["mr_wc.py"])

with mr_job.make_runner() as runner:
    runner.run()
    for line in runner.stream_output():
        print mr_job.parse_output_line(line)

(u'words', 102)
(u'chars', 987)
(u'lines', 28)
